<a href="https://colab.research.google.com/github/cityardhelaalisya/Data-Analyst-Portofolio/blob/main/Retail_Store_Sales_Exploratory_Data_Analyst_and_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [95]:
#load  the retail store sales dataset
df = pd.read_csv('/content/retail_store_sales.csv')

#shows the dataset
df.head(10)

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False
5,TXN_7482416,CUST_09,Patisserie,NaN,NaN,10.0,200.0,Credit Card,Online,2023-11-30,NaN
6,TXN_3652209,CUST_07,Food,Item_1_FOOD,5.0,8.0,40.0,Credit Card,In-store,2023-06-10,True
7,TXN_1372952,CUST_21,Furniture,NaN,33.5,NaN,NaN,Digital Wallet,In-store,2024-04-02,True
8,TXN_9728486,CUST_23,Furniture,Item_16_FUR,27.5,1.0,27.5,Credit Card,In-store,2023-04-26,False
9,TXN_2722661,CUST_25,Butchers,Item_22_BUT,36.5,3.0,109.5,Cash,Online,2024-03-14,False


In [96]:
#info dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


In [97]:
#check null values
missing_val = df.isnull().sum()[df.isnull().sum() > 0]
df_missing_val = pd.DataFrame({'NaN Counts': missing_val,
                               'NaN Percentages' : missing_val/len(df) * 100,
                               'Data Type' : df.dtypes}).sort_values(by='NaN Percentages', ascending=False)


df_missing_val
# There are 4 columns with null values. there are 'Discount Applied', 'Item', 'Price Per Unit', 'Quantity', 'Total Spent'
# For that columns, there is no column with NaN percentages above 90%. So we don't need to drop the column and fill null values with mod if data types = category and median or mean if data types = int/float

,NaN Counts,NaN Percentages,Data Type
Discount Applied,4199.0,33.391650,object
Item,1213.0,9.646123,object
Price Per Unit,609.0,4.842942,float64
Quantity,604.0,4.803181,float64
Total Spent,604.0,4.803181,float64
Category,NaN,NaN,object
Customer ID,NaN,NaN,object
Location,NaN,NaN,object
Payment Method,NaN,NaN,object
Transaction Date,NaN,NaN,object


In [98]:
# check datas in Discount Applied
df['Discount Applied'].unique()

# fill null values with mode
df['Discount Applied'] = df['Discount Applied'].fillna(df['Discount Applied'].mode()[0])
# column discount applied has successfully filled null value with mode values = 'True'

/tmp/ipython-input-3373053438.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Discount Applied'] = df['Discount Applied'].fillna(df['Discount Applied'].mode()[0])


np.int64(0)

In [99]:
# check datas in column Item
df['Item'].unique()

# There are no unconsistent datas but null values exist. Fill null value with mode
mode_Item = df['Item'].mode()[0]

# fill null values with mode values = 'Item_2_BEV'
df['Item'] = df['Item'].fillna(mode_Item)

np.int64(0)

In [101]:
# because price per unit, quantity and total spent are numerice data types. I want to check whether we can fillna with calculate each other
df[(df['Price Per Unit'].isna()) & (df['Quantity'].notna()) & (df['Total Spent'].notna())][['Price Per Unit', 'Quantity', 'Total Spent']]

# we can calculate and fill na values in Price per Unit with Total Spent / Quantity
cal_price = df['Total Spent'] / df['Quantity']
df['Price Per Unit'] = df['Price Per Unit'].fillna(cal_price)
df['Price Per Unit'].isna().sum()

# check Quantity and Total Spent column
df[(df['Price Per Unit'].notna()) & (df['Quantity'].isna()) & (df['Total Spent'].notna())][['Price Per Unit', 'Quantity', 'Total Spent']]
df[(df['Price Per Unit'].notna()) & (df['Quantity'].isna()) & (df['Total Spent'].isna())][['Price Per Unit', 'Quantity', 'Total Spent']]

# we can't use fillna values with calculate 'Price Per Unit' and 'Total Spent'. So we need fill it with median or median use bloxplot
df.describe()

,Price Per Unit,Quantity,Total Spent
count,12575.000000,11971.000000,11971.000000
mean,23.369304,5.536380,129.652577
std,10.748728,2.857883,94.750697
min,5.000000,1.000000,5.000000
25%,14.000000,3.000000,51.000000
50%,23.000000,6.000000,108.500000
75%,33.500000,8.000000,192.000000
max,41.000000,10.000000,410.000000


In [102]:
# fill na values for 'Quantity' with mean and 'Total Spent' with median
df['Quantity'] = df['Quantity'].fillna(df['Quantity'].mean())
df['Total Spent'] = df['Total Spent'].fillna(df['Total Spent'].median())

df.isna().sum()

,0
Transaction ID,0
Customer ID,0
Category,0
Item,0
Price Per Unit,0
Quantity,0
Total Spent,0
Payment Method,0
Location,0
Transaction Date,0
